In [1]:
from datasets import load_dataset, DatasetDict, Audio
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate
import pandas as pd

In [ ]:
# Выбор переменных для обучения
datasets = "mozilla-foundation/common_voice_11_0"   # Из какого датасета берем данные для обучения
lang_datasets = "ru"                                # Выбираем язык датасета
wisp_model = "openai/whisper-small"                 # Выбираем базовую модель
wisp_lang = "Russian"                               # Выбираем язык базовой модели
output_dir = "./whisper-small-ru"                   # Выбираем путь сохранения результатов обучения

# Подготовка данных и загрузка датасета

In [ ]:
common_voice = DatasetDict()

common_voice["train"] = load_dataset(datasets, lang_datasets, split="train+validation")
common_voice["test"] = load_dataset(datasets, lang_datasets, split="test")

print(common_voice)

f:\prog\anaconda\envs\fpmi_py\lib\site-packages\datasets\load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 32491
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 9630
    })
})


In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 32491
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 9630
    })
})


# Подготовка процессора, токенизатора и экстрактора признаков

In [ ]:
# Предварительно обрабатывавем входные данные
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
# Обрабатываем выходные данные модели в текстовый формат
tokenizer = WhisperTokenizer.from_pretrained(wisp_model, language=wisp_lang, task="transcribe")

In [ ]:
# Наследует токенизатор и экстрактор - нужно для работы модели
processor = WhisperProcessor.from_pretrained(wisp_model, language=wisp_lang, task="transcribe")

Согласуем частоту дискретизации нашего аудио с частотой дискретизации модели(16 кГц)

In [ ]:
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': 'C:\\Users\\Dronxix\\.cache\\huggingface\\datasets\\downloads\\extracted\\bbacf3b2c63dcd87076bb896cdd00714ee4220b4078baf7a1f4d68a374bf7740\\ru_train_0/common_voice_ru_26426765.mp3', 'array': array([-5.68434189e-14, -1.81898940e-12, -1.70530257e-12, ...,
        9.95262781e-07, -1.48648405e-06, -2.20581842e-06]), 'sampling_rate': 16000}, 'sentence': 'Демократия неумолимо продвигается по Африке, и «арабская весна» была ее кульминацией.'}


In [ ]:
def prepare_dataset(batch):
    from transformers import WhisperFeatureExtractor, WhisperTokenizer
    feature_extractor = WhisperFeatureExtractor.from_pretrained(wisp_model)
    tokenizer = WhisperTokenizer.from_pretrained(wisp_model, language="Russian", task="transcribe")

    # Загружаем предобработанное аудио
    audio = batch["audio"]

    # Вычисляем входные признаки логарифмической спектрограммы Mel из аудиомассива.
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # Кодируем текст в идентификаторы меток
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=8)

# Создаем сборщика данных

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Разделяем входные признаки и лэйблы
        # Возвращаем pythorch тензоры для признаков
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Получаем токенизированные лейблы
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # Увеличиваем через .pad лэйблы домаксимальной длинны
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # Заменяемм паддинги на -100, что бы не учитывать лейблы при вычислении потерь
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # Затем вырезаем токен начала транскрипта из начала последовательности лэйблов,
        # так как мы добавляем его позже во время обучения.
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
# Инициализируем сборщика данных
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# Метрики оценки

In [ ]:
# Загрузка метрики WER

metric = evaluate.load("wer")

In [ ]:
# Функция расчета метрик
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Заменяем -100 на pad_token_id в label_ids
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# Загрузка предобученной модели

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained(wisp_model)

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
# Задаем аргументы для обучения модели
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,  # Название папки куда сохранить веса и чекпоинты
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [ ]:
# Тренер для обучения модели
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
# Сохраняем предобученное состояние модели
processor.save_pretrained(training_args.output_dir)

# Запуск обучения

In [ ]:
# Запускаем обучение
trainer.train()

  0%|          | 0/4000 [00:00<?, ?it/s]

f:\prog\anaconda\envs\fpmi_py\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


{'loss': 2.929, 'learning_rate': 4.2000000000000006e-07, 'epoch': 0.01}
{'loss': 2.1936, 'learning_rate': 9.200000000000001e-07, 'epoch': 0.02}
{'loss': 1.4827, 'learning_rate': 1.42e-06, 'epoch': 0.04}
{'loss': 0.8146, 'learning_rate': 1.9200000000000003e-06, 'epoch': 0.05}
{'loss': 0.5925, 'learning_rate': 2.42e-06, 'epoch': 0.06}
{'loss': 0.47, 'learning_rate': 2.92e-06, 'epoch': 0.07}
{'loss': 0.4253, 'learning_rate': 3.4200000000000007e-06, 'epoch': 0.09}
{'loss': 0.3741, 'learning_rate': 3.920000000000001e-06, 'epoch': 0.1}
{'loss': 0.3053, 'learning_rate': 4.42e-06, 'epoch': 0.11}
{'loss': 0.2358, 'learning_rate': 4.92e-06, 'epoch': 0.12}
{'loss': 0.2279, 'learning_rate': 5.420000000000001e-06, 'epoch': 0.14}
{'loss': 0.2073, 'learning_rate': 5.92e-06, 'epoch': 0.15}
{'loss': 0.1829, 'learning_rate': 6.42e-06, 'epoch': 0.16}
{'loss': 0.2112, 'learning_rate': 6.92e-06, 'epoch': 0.17}
{'loss': 0.2195, 'learning_rate': 7.420000000000001e-06, 'epoch': 0.18}
{'loss': 0.1959, 'learnin

  0%|          | 0/1204 [00:00<?, ?it/s]

{'eval_loss': 0.21552051603794098, 'eval_wer': 43.83486372960828, 'eval_runtime': 5786.9935, 'eval_samples_per_second': 1.664, 'eval_steps_per_second': 0.208, 'epoch': 0.49}


f:\prog\anaconda\envs\fpmi_py\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.1722, 'learning_rate': 8.511428571428571e-06, 'epoch': 0.5}
{'loss': 0.1749, 'learning_rate': 8.44e-06, 'epoch': 0.52}
{'loss': 0.1768, 'learning_rate': 8.36857142857143e-06, 'epoch': 0.53}
{'loss': 0.1606, 'learning_rate': 8.297142857142859e-06, 'epoch': 0.54}
{'loss': 0.1773, 'learning_rate': 8.225714285714288e-06, 'epoch': 0.55}
{'loss': 0.1613, 'learning_rate': 8.154285714285715e-06, 'epoch': 0.57}
{'loss': 0.1774, 'learning_rate': 8.082857142857144e-06, 'epoch': 0.58}
{'loss': 0.1626, 'learning_rate': 8.011428571428573e-06, 'epoch': 0.59}
{'loss': 0.1716, 'learning_rate': 7.94e-06, 'epoch': 0.6}
{'loss': 0.1736, 'learning_rate': 7.86857142857143e-06, 'epoch': 0.62}
{'loss': 0.1684, 'learning_rate': 7.797142857142858e-06, 'epoch': 0.63}
{'loss': 0.1522, 'learning_rate': 7.725714285714286e-06, 'epoch': 0.64}
{'loss': 0.1673, 'learning_rate': 7.654285714285715e-06, 'epoch': 0.65}
{'loss': 0.1667, 'learning_rate': 7.5828571428571444e-06, 'epoch': 0.66}
{'loss': 0.155, 'lear

  0%|          | 0/1204 [00:00<?, ?it/s]

{'eval_loss': 0.19007627665996552, 'eval_wer': 34.499853472697076, 'eval_runtime': 5903.9872, 'eval_samples_per_second': 1.631, 'eval_steps_per_second': 0.204, 'epoch': 0.98}


PermissionError: [WinError 32] Процесс не может получить доступ к файлу, так как этот файл занят другим процессом: './whisper-small-ru\\tmp-checkpoint-2000' -> './whisper-small-ru\\checkpoint-2000'

# Выводим результаты обучения

In [ ]:
# Функция сбора WER
import os
from pathlib import Path
import json
check_list = []
wer_list = []
df = pd.DataFrame()
for i in Path(output_dir).rglob('*.json'):
    if 'trainer_state' in str(i):
        f = open(os.path.abspath(i))
        data = json.load(f)
        check_list.append(data['global_step'])
        wer_list.append(data['best_metric'])
df['Checkpoint'] = check_list
df['WER'] = wer_list

In [ ]:
# Выводим результат обучения
df

,Checkpoint,WER
0,1000,43.834864
1,2000,34.499853
2,3000,31.703624
3,4000,31.703624
